In [ ]:
import pandas as pd
from bokeh.plotting import output_notebook, figure, show, ColumnDataSource
from bokeh.models import HoverTool, SingleIntervalTicker
from bokeh.palettes import Pastel1_3, Paired6

output_notebook()

In [ ]:
df = pd.read_csv('wordnet.csv', delimiter='\t')
df

In [ ]:
df['x_labels'] = df['decay_rate'].astype(str) + ', ' + df['depth'].astype(str)
df['stimuli_x'] = df.index + 3/4
df['stimuli_y'] = df['avg_stimuli'] / 2
df['lure_x'] = df.index + 3/4 + 1/4
df['lure_y'] = df['avg_lure'] / 2
df['external_x'] = df.index + 3/4 + 2/4
df['external_y'] = df['avg_external'] / 2

# create figure
f = figure(width=900, height=300,
           x_range=list(df['x_labels']),
           y_range=[0, 1],
)
f.xaxis.axis_label = 'decay rate, depth'
f.xgrid.grid_line_color = None
f.yaxis.axis_label = 'Recall Rate'
f.ygrid[0].ticker.desired_num_ticks = 10
f.yaxis.minor_tick_line_color = None

color_map = {
    'stimuli':Pastel1_3[1],
    'lure':Pastel1_3[0],
    'external':Pastel1_3[2],
}

# plot data bars
for col in ('stimuli', 'lure', 'external'):
    columns = ['decay_rate', 'depth', col + '_x', col + '_y', 'avg_' + col]
    plot_df = df[columns].rename(columns={
            columns[2]:'x',
            columns[3]:'y',
            columns[4]:'height',
    })
    renderer = f.rect(
        x='x',
        y='y',
        width=1/4,
        height='height',
        color=color_map[col],
        legend=col.capitalize(),
        source=ColumnDataSource(plot_df)
    )
    f.add_tools(HoverTool(renderers=[renderer], tooltips=[
            ('Decay Rate', '@decay_rate{0.00}'),
            ('Spreading Depth', '@depth'),
            ('Type', col.capitalize()),
            ('Recall Rate', '@height{0.00%}'),
    ]))

# plot error bars
for i, col in enumerate(['stimuli', 'external']):
    columns = [col + '_x', col + '_y', 'avg_' + col, 'std_' + col]
    plot_df = df[columns].rename(columns={
            columns[0]:'x',
            columns[1]:'y',
            columns[2]:'height',
            columns[3]:'stdev',
    })
    plot_df['top'] = plot_df['height'] + plot_df['stdev']
    plot_df['bottom'] = plot_df['height'] - plot_df['stdev']
    plot_df['left'] = plot_df['x'] - 1/8
    plot_df['right'] = plot_df['x'] + 1/8
    f.square(x='x', y='height', color='#888A85', source=ColumnDataSource(plot_df))
    f.segment(x0='left', y0='top', x1='right', y1='top', color='#888A85', source=ColumnDataSource(plot_df))
    f.segment(x0='left', y0='bottom', x1='right', y1='bottom', color='#888A85', source=ColumnDataSource(plot_df))
    f.segment(x0='x', y0='top', x1='x', y1='bottom', color='#888A85', source=ColumnDataSource(plot_df))
    
# plot human data
f.line(
    x=[0, df.shape[0] + 1],
    y=0.62,
    line_width=3,
    color=Paired6[1],
    legend='DRM Stimuli',
)
f.line(
    x=[0, df.shape[0] + 1],
    y=0.55,
    line_width=3,
    color=Paired6[5],
    legend='DRM Lure',
)

f.legend.location = 'top_left'

show(f)